In [1]:
import numpy as np # para aleatorio
from scipy.ndimage.filters import gaussian_filter1d as gaussf
import pandas as pd

In [2]:
import pyratbay as pb
import pyratbay.constants as pc

ModuleNotFoundError: No module named 'pyratbay'

In [3]:
comp_no_bio = ['CO2','CO','H2O', 'CH4', 'N2'] # Elementos de firma no biologica
comp_bio = ['CH4', 'N2O', 'O3','CH3Cl'] # Elementos de firma biologica
atm = "ATM/atm_prod.cfg" # selecciona el doc a modificar

In [62]:
spect = "/work/ExoSpectrum/SPECTRUM/spectrum_transmission.cfg"
hitran = "/work/ExoSpectrum/HITRAN/tli_hitran.cfg"
atm = "/work/ExoSpectrum/ATM/atm_prod.cfg"

In [ ]:
# ---- BD HITRAN ----
pyrat_hitran = pb.run(hitran)

In [31]:
def comp(number_bio,number_not_bio):
    composition = []
    comp_bio = ['H2', 'CH4', 'N2O', 'N2', 'O3', 'CH3Cl', 'N2O']
    comp_no_bio = ['CO2', 'CO', 'H2O']
    ch_bio = np.random.choice(comp_bio,size=number_bio,replace=False)
    ch_nbio = np.random.choice(comp_no_bio,size=number_not_bio,replace=False)
    composition = np.array([ [i,j] for i in ch_bio for j in ch_nbio])
    return np.unique(composition.flatten())

In [36]:
def abundance(num,imin,imax):
    l = []
    suma = 0 # suma acumulativa (debe ser menor que 1.0)
    while(suma < 0.98 and len(l) != num):
        if len(l) == 0: # primer valor
            r1 = np.random.rand()
            if r1 > imin and r1 < imax : # debe ser un min < r1 < max, para controlar un valor con gran concentracion
                l.append(r1)
                suma += l[-1] # suma el primer elemento.
        elif len(l) != 0: 
            r = np.random.randint(1,11) # selecciona un numero aleatorio entre 1 y 10
            n = l[-1]/r # divide el ultimo numero entre ese numero aleatorio
            if (sum(l) + n) < 1.0 and n != l[-1]: # es la suma de numeros creados mas este ultimo menor que 1?
                #print(sum(l) + n)
                l.append(n) # almacenar este numero
                suma += n  # sumar el nuevo elemento
    return l

def corr_ab(num):
    flag = False
    while(flag == False): # a veces se obtiene sumas menores a 0.95 y listas menores a num
        ab = abundance(num,0.85,0.90)
        if len(ab) == num and sum(ab) > 0.95: # si el resultado tiene un valor mayor de 0.95 y un numero correcto de longitud
            flag = True # se acepta
    #print(sum(ab))
    ab_corr = [str(format(elem,'.1E')) for elem in ab] # convierte en string y en notacion cientifica
    return ab_corr


In [60]:
def write(comp, abun):
    cm = ' '.join(comp) # junta las listas en un string
    ab = ' '.join(abun)
    with open(atm,"r") as rd:
        data = rd.readlines() # obtiene la lista de string por linea en el doc
        
    data[22] = f'species = H2 {cm}\n' # sobreescribe los elementos por el seleccionado aleatoriamente
    data[23] = f'uniform = {ab}\n' # sobreescribe los porcentajes por los seleccionados aleatoriamente
    conct = ''.join(data) # junta de nuevo toda la lista ya modificada

    with open("ATM/atm_prod.cfg","w") as wd: # escribe en el texto
        wd.write(conct)


In [ ]:
for i in range(1):
    
    # selecciona aleatoriamente 4 elementos de la lista de ejemplos (se tiene H2 por defecto como el más abundante)
    comp4 = comp(3,1) # (numero comp biologicos, numero comp no biologicos)
    abun4 = corr_ab(4) # llama la generacion de numeros.
    
    # Genera el archivo .atm
    write(comp4,abun4)
    
    # --- atmósfera ----
    pyrat_atm = pb.run(atm)
    
    # --- espectro ---
    pyrat = pb.run(spect)
    
    wl = 1.0 / (pyrat.spec.wn*pc.um)
    depth = pyrat.spec.spectrum / pc.percent
    
    df = pd.DataFrame({'wl': wl, 'p': depth})
    df.to_csv(f"Resultados/no_bio_{comp4}_{abun4}.txt",sep="\t",index=None)
    

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=589cf5e2-2262-4138-b15b-c07c0b65a473' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>